# Second & Third Part

### 1. Data preprocessing
In this part, the main goal is to seperate training and testing data, filter out useless information, and combine tweets with emotions as labels.

#### (1) Observe data_identification.csv to seperate train and test identifications

The total number of tweet_id is 1867535. From len() we verify that we successfully seperate training and testing data.

In [2]:
import json
import pandas as pd

In [2]:
#data_identification.csv
train_test_set = pd.read_csv("dm19-lab2-nthu/data_identification.csv",
                         header=None, names=["id", "identification"])

In [3]:
train_test_set.head()

,id,identification
0,tweet_id,identification
1,0x28cc61,test
2,0x29e452,train
3,0x2b3819,train
4,0x2db41f,test


In [4]:
#drop the first title row
train_test_set = train_test_set.drop([0], axis=0)

In [5]:
train_test_set.head()

,id,identification
1,0x28cc61,test
2,0x29e452,train
3,0x2b3819,train
4,0x2db41f,test
5,0x2a2acc,train


In [50]:
train_ids = train_test_set[train_test_set['identification']=='train'].filter(['id'])

In [51]:
test_ids = train_test_set[train_test_set['identification']=='test'].filter(['id'])

In [52]:
train_ids[:10]

,id
2,0x29e452
3,0x2b3819
5,0x2a2acc
6,0x2a8830
7,0x20b21d
8,0x2452cf
9,0x2d729d
10,0x2ab56d
11,0x1f3657
12,0x1fcc53


In [57]:
len(train_ids)

1455563

In [58]:
len(test_ids)

411972

#### (2) Observe emotion.csv

From len(), I found that emotion.csv only has train_ids since the number of datas in emotion_set is same as train_ids. Thus, It's no need to do more process on emotion.csv but just read and save as pickle.

In [22]:
#emotion.csv
emotion_set = pd.read_csv("dm19-lab2-nthu/emotion.csv",
                         header=None, names=["id", "emotion"])

In [23]:
#drop the first title row
emotion_set = emotion_set.drop([0], axis=0)

In [24]:
emotion_set[:10]

,id,emotion
1,0x3140b1,sadness
2,0x368b73,disgust
3,0x296183,anticipation
4,0x2bd6e1,joy
5,0x2ee1dd,anticipation
6,0x34cd80,joy
7,0x33f099,sadness
8,0x2ae7b7,sadness
9,0x2408d4,trust
10,0x2b193b,sadness


In [25]:
len(emotion_set)

1455563

In [26]:
## save to pickle file
emotion_set.to_pickle("train_df.pkl") 

#### (3) Observe tweets_DM.json, get texts and tweets_id from raw data, and combine training data with emotion labels

Since tweets_DM.json includes both train and test data, I use train_ids above to divide the file into two parts. In addition to seperating the file, I also merge emotion_set with train_tweet_df for further works.

In [27]:
#tweets_DM.json
tweets_df = pd.read_json("dm19-lab2-nthu/tweets_DM.json", lines=True)

In [28]:
tweets_df.head()

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [30]:
tweets_source_df = tweets_df['_source']

In [31]:
tweets_df['text'] = tweets_source_df.apply(lambda x: x['tweet']['text'])

In [32]:
tweets_df['id'] = tweets_df['_source'].apply(lambda x: x['tweet']['tweet_id'])

In [33]:
filter_tweets_df = tweets_df.filter(['text','id'])

In [34]:
filter_tweets_df.head()

,text,id
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,"Confident of your obedience, I write to you, k...",0x28b412
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,"""Trust is not the same as faith. A friend is s...",0x2de201


In [53]:
train_tweets_df = filter_tweets_df[filter_tweets_df['id'].isin(train_ids['id'])]

In [55]:
test_tweets_df = filter_tweets_df[~filter_tweets_df['id'].isin(train_ids['id'])]

In [54]:
len(train_tweets_df)

1455563

In [56]:
len(test_tweets_df)

411972

In [59]:
train_tweets_df.head()

,text,id
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8


In [60]:
test_tweets_df.head()

,text,id
2,"Confident of your obedience, I write to you, k...",0x28b412
4,"""Trust is not the same as faith. A friend is s...",0x2de201
9,When do you have enough ? When are you satisfi...,0x218443
30,"God woke you up, now chase the day #GodsPlan #...",0x2939d5
33,"In these tough times, who do YOU turn to as yo...",0x26289a


In [ ]:
#add emotion labels to train dataframe
train_tweets_df = train_tweets_df.merge(emotion_set)

In [62]:
## save to pickle file
train_tweets_df.to_pickle("train_tweets_df.pkl") 
test_tweets_df.to_pickle("test_tweets_df.pkl")

### 2. Feature engineering and Model Building
Since feature engineering is related to model building, I would like to introduce both them in once.

### A. Combined pre-train w2v model with scikit learn machine learning model
Each tweet consists of many words, so I think it's a good start to create "sentence vectors" as features by w2v model. After finishing feature engineering, I build some models by traditional machine learning.

#### (1) Load GoogleNews vectors

In [63]:
from gensim.models import KeyedVectors
from gensim.utils import tokenize
from gensim.models import Word2Vec, Phrases

## Note: this model is very huge, this will take some time ...
model_path = "GoogleNews/GoogleNews-vectors-negative300.bin.gz"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print('load ok')

c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


load ok


#### (2) Load train and test data

In [64]:
train_tweets_df = pd.read_pickle("train_tweets_df.pkl")
test_tweets_df = pd.read_pickle("test_tweets_df.pkl")

#### (3) Tokenize and use w2v

We tokenize and transform each sentence into one vector by summing and averaging all word vectors in each sentence.

In [65]:
import nltk
import numpy as np

In [66]:
def tokenize_text(sentence):
    tokens = nltk.wordpunct_tokenize(sentence.lower())
    return tokens

In [68]:
# tokenize texts using tokenize_text()
train_tweets_df['unigrams'] = train_tweets_df['text'].apply(lambda x: tokenize_text(x))

In [69]:
# tokenize texts using tokenize_text()
test_tweets_df['unigrams'] = test_tweets_df['text'].apply(lambda x: tokenize_text(x))

In [78]:
train_tweets_df.head()

,text,id,unigrams
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha..."
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan..."
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]"
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th..."
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ..."


In [70]:
#save w2v index2word to a set
index2word_set = set(w2v_google_model.wv.index2word)

In [74]:
def w2v(tokens):
    num_of_word = 0
    sentence_vec = np.zeros(300)
    avg_vec = np.zeros(300)
    for word in tokens:
        if word not in index2word_set:
            continue
        else:
            sentence_vec = np.add(sentence_vec,w2v_google_model[word])
            num_of_word += 1
        if(num_of_word>0):
            length = np.linalg.norm(sentence_vec)
            avg_vec = sentence_vec/length
    return avg_vec

In [79]:
# translate unigrams to vector using w2v()
train_tweets_df['vector'] = train_tweets_df['unigrams'].apply(lambda x: w2v(x))

In [80]:
test_tweets_df['vector'] = test_tweets_df['unigrams'].apply(lambda x: w2v(x))

In [81]:
train_tweets_df.head()

,text,id,unigrams,vector
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha...","[-0.04398071877257593, 0.005973617156422666, 0..."
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan...","[-0.07402511888368858, -0.027571952599088873, ..."
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]","[-0.08825499574715533, 0.021097196412366435, 0..."
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th...","[-0.05885132318051363, -0.02492928390044968, 0..."
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ...","[-0.01618322244071664, -0.02359793926091037, 0..."


In [83]:
#add emotion labels to train dataframe
train_tweets_df = train_tweets_df.merge(emotion_set)

In [84]:
train_tweets_df.head()

,text,id,unigrams,vector,emotion
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha...","[-0.04398071877257593, 0.005973617156422666, 0...",anticipation
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan...","[-0.07402511888368858, -0.027571952599088873, ...",sadness
2,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]","[-0.08825499574715533, 0.021097196412366435, 0...",fear
3,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th...","[-0.05885132318051363, -0.02492928390044968, 0...",joy
4,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ...","[-0.01618322244071664, -0.02359793926091037, 0...",anticipation


In [86]:
test_tweets_df.head()

,text,id,unigrams,vector
2,"Confident of your obedience, I write to you, k...",0x28b412,"[confident, of, your, obedience, ,, i, write, ...","[-0.013770567964291183, -0.03728248351083156, ..."
4,"""Trust is not the same as faith. A friend is s...",0x2de201,"["", trust, is, not, the, same, as, faith, ., a...","[0.030071268925277977, -0.036261832915662436, ..."
9,When do you have enough ? When are you satisfi...,0x218443,"[when, do, you, have, enough, ?, when, are, yo...","[0.0605162729192968, -0.0009634782860012661, 0..."
30,"God woke you up, now chase the day #GodsPlan #...",0x2939d5,"[god, woke, you, up, ,, now, chase, the, day, ...","[0.01819009581685019, 0.001476407360784927, 0...."
33,"In these tough times, who do YOU turn to as yo...",0x26289a,"[in, these, tough, times, ,, who, do, you, tur...","[0.017859831789070642, 0.09871569549570929, 0...."


In [85]:
#filter useful columns
train_tweets_df = train_tweets_df.filter(['id','vector','emotion'])

In [87]:
test_tweets_df = test_tweets_df.filter(['id','vector'])

In [88]:
#drop nan value in train and replace nan value with all zero vector in test
train_tweets_df = train_tweets_df.dropna()

In [89]:
len(train_tweets_df)

1455563

#### (4) Train models and predict the testing data

Since Naive Bayes couldn't train negtive values, we use linear SVC and logistic regression to train.

In [15]:
import numpy as np

In [62]:
#We couldn't save train_tweets_vec_df.pkl by jupyter since the kernal will be crashed, We save the files in local environment
train_tweets_df = pd.read_pickle("train_tweets_vec_df.pkl")

In [63]:
#prepare X and Y training data and label
X_train = train_tweets_df['vector'][:int(len(train_tweets_df)*3/4)]
Y_train = train_tweets_df['emotion'][:int(len(train_tweets_df)*3/4)]

In [64]:
X_train = np.array(X_train)

In [17]:
len(X_train)

1091672

In [18]:
len(Y_train)

1091672

In [65]:
#also prepare X testing data
X_test = train_tweets_df['vector'][int(len(train_tweets_df)*3/4):]
Y_test = train_tweets_df['emotion'][int(len(train_tweets_df)*3/4):]

In [66]:
X_test = np.array(X_test)

In [21]:
from sklearn.svm import LinearSVC

In [22]:
## build LinearSVC model
LSVC_model = LinearSVC()

## training!
LSVC_model = LSVC_model.fit(list(X_train), Y_train)

## predict!
y_train_pred = LSVC_model.predict(list(X_train))
y_test_pred = LSVC_model.predict(list(X_test))

In [24]:
import pickle
with open('LSVC_clf.pickle', 'wb') as f:
    pickle.dump(LSVC_model, f)

In [25]:
## accuracy
from sklearn.metrics import accuracy_score
acc_train = accuracy_score(y_true=Y_train,y_pred=y_train_pred)
acc_test = accuracy_score(y_true=Y_test,y_pred=y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

training accuracy: 0.46
testing accuracy: 0.46


In [67]:
from sklearn.linear_model import LogisticRegression

In [68]:
## build LogisticRegression model
LR_model = LogisticRegression()

## training!
LR_model = LR_model.fit(list(X_train), Y_train)

In [69]:
import pickle
with open('LR_clf.pickle', 'wb') as f:
    pickle.dump(LR_model, f)

In [70]:
## predict!
y_train_pred = LR_model.predict(list(X_train))
y_test_pred = LR_model.predict(list(X_test))

In [71]:
## accuracy
from sklearn.metrics import accuracy_score
acc_train = accuracy_score(y_true=Y_train,y_pred=y_train_pred)
acc_test = accuracy_score(y_true=Y_test,y_pred=y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

training accuracy: 0.47
testing accuracy: 0.47


#### (5) write submission file

In [72]:
test_tweets_df = pd.read_pickle("test_tweets_vec_df.pkl")

In [27]:
with open('LSVC_clf.pickle', 'rb') as f:
    LSVC_model = pickle.load(f)

In [28]:
X_unknown = test_tweets_df['vector']
X_unknown = np.array(X_unknown)
y_unknown_pred = LSVC_model.predict(list(X_unknown))

In [29]:
predict_test = {
    "id":list(test_tweets_df['id']),
    "emotion":list(y_unknown_pred)
}

In [32]:
predict_test = pd.DataFrame(predict_test)
predict_test.to_csv('result.csv',index=False)

In [73]:
with open('LR_clf.pickle', 'rb') as f:
    LR_model = pickle.load(f)

In [74]:
X_unknown = test_tweets_df['vector']
X_unknown = np.array(X_unknown)
y_unknown_pred = LR_model.predict(list(X_unknown))

In [75]:
predict_test = {
    "id":list(test_tweets_df['id']),
    "emotion":list(y_unknown_pred)
}

In [76]:
predict_test = pd.DataFrame(predict_test)
predict_test.to_csv('LR_result.csv',index=False)

#### (6) result
I use 3/4 of training data to build two models, and it almost approached the baseline by logistic regression. I think the reason why they couldn't reach the baseline is because I use a rough way to feature sentences. Averaging all word vectors may not be able to include the meaning of context and the natural relationship between attributes and true labels.

### B. Deep Learning

Since traditional machine learning with w2v couldn't get high score, I try to use deep learning.

#### For 300 features:
#### (1) Transform sentences into 300 features bag-of-words.
Though the example in DM19_Lab2_Master uses 500 features, I consider if less features will improve the model or not. Therefore, I set max_features = 300 in first try. Besides, I only use 500k of data to train to avoid from overfitting.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
train_df = pd.read_pickle("train_tweets_df.pkl")

In [35]:
test_df = pd.read_pickle("test_tweets_df.pkl")

In [36]:
import nltk
BOW_vectorizer = CountVectorizer(max_features=300, tokenizer=nltk.word_tokenize)

In [37]:
num_of_train = 500000
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(train_df['text'][:num_of_train])

# 2. Transform documents to document-term matrix.
X_train = BOW_vectorizer.transform(train_df['text'][:num_of_train])
y_train = train_df['emotion'][:num_of_train]
X_test = BOW_vectorizer.transform(train_df['text'][1400000:])
y_test = train_df['emotion'][1400000:]

#### (2) Build neural network

In [38]:
import keras

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

Using TensorFlow backend.


X_train.shape:  (500000, 300)
y_train.shape:  (500000,)
X_test.shape:  (55563, 300)
y_test.shape:  (55563,)


In [39]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:7]:\n', y_train[0:7])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:7]:
 0    anticipation
1         sadness
2            fear
3             joy
4    anticipation
5             joy
6         sadness
Name: emotion, dtype: object

y_train.shape:  (500000,)
y_test.shape:  (55563,)


## After convert
y_train[0:7]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (500000, 8)
y_test.shape:  (55563, 8)


In [40]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  300
output_shape:  8


In [41]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                19264     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8)                 0         
Total

#### (3) Train the network

In [42]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')
model.save('BOW300_500k.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 500000 samples, validate on 55563 samples
Epoch 1/25





500000/500000 [==============================] - 31s 63us/step - loss: 1.4958 - acc: 0.4539 - val_loss: 1.4674 - val_acc: 0.4599
Epoch 2/25
500000/500000 [==============================] - 31s 62us/step - loss: 1.4442 - acc: 0.4727 - val_loss: 1.4493 - val_acc: 0.4682
Epoch 3/25
500000/500000 [==============================] - 32s 65us/step - loss: 1.4295 - acc: 0.4778 - val_loss: 1.4430 - val_acc: 0.4698
Epoch 4/25
500000/500000 [==============================] - 31s 62us/step - loss: 1.4209 - acc: 0.4812 - val_loss: 1.4416 - val_acc: 0.4715
Epoch 5/25
500000/500000 [==============================] - 30s 61us/step - loss: 1.4148 - acc: 0.4825 - val_loss: 1.4420 - val_acc: 0.4728
Epoch 6/25
500000/500000 [==============================] - 29s 58us/step - loss: 1.4103 - acc: 0.4843 - val_loss: 1.4429 - val_acc: 0.4716
Epoch 7

In [48]:
y_test_pred = model.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), label_decode(label_encoder,y_test_pred)), 2)))

testing accuracy: 0.47


c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### (4)  Predict and write submission

In [ ]:
X_unknown = BOW_vectorizer.transform(test_df['text'])

In [44]:
y_unknown_pred = model.predict(X_unknown)

In [45]:
y_unknown_pred = label_decode(label_encoder, y_unknown_pred)

c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [46]:
predict_test = {
    "id":list(test_df['id']),
    "emotion":list(y_unknown_pred)
}

In [47]:
predict_test = pd.DataFrame(predict_test)
predict_test.to_csv('BOW300_500k.csv',index=False)

The first deep learning model couldn't reach the benchmark, so I need to adjust some parts.

#### (5) resume the training process

The val_acc above seems to glue on about 0.47. I think this may because the number of epochs isn't enough, so I resume the training process in order to improve the val_acc. Below is the second training process.

In [53]:
from keras.models import load_model
new_model = load_model('BOW300_500k.h5')

In [54]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = new_model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')
model.save('second_BOW300_500k.h5')

Train on 500000 samples, validate on 55563 samples
Epoch 1/25
500000/500000 [==============================] - 37s 74us/step - loss: 1.3837 - acc: 0.4945 - val_loss: 1.4546 - val_acc: 0.4715
Epoch 2/25
500000/500000 [==============================] - 34s 68us/step - loss: 1.3833 - acc: 0.4948 - val_loss: 1.4534 - val_acc: 0.4704
Epoch 3/25
500000/500000 [==============================] - 36s 72us/step - loss: 1.3825 - acc: 0.4952 - val_loss: 1.4551 - val_acc: 0.4708
Epoch 4/25
500000/500000 [==============================] - 34s 67us/step - loss: 1.3821 - acc: 0.4955 - val_loss: 1.4560 - val_acc: 0.4724
Epoch 5/25
500000/500000 [==============================] - 34s 68us/step - loss: 1.3814 - acc: 0.4957 - val_loss: 1.4565 - val_acc: 0.4695
Epoch 6/25
500000/500000 [==============================] - 34s 68us/step - loss: 1.3817 - acc: 0.4958 - val_loss: 1.4565 - val_acc: 0.4713
Epoch 7/25
500000/500000 [==============================] - 35s 70us/step - loss: 1.3811 - acc: 0.4958 - val_

In [55]:
y_test_pred = model.predict(X_test)

In [56]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), label_decode(label_encoder,y_test_pred)), 2)))

testing accuracy: 0.47


c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### (5) result
More epochs can only improve train accuracy but not val accuracy. This means that more epochs will cause overfitting, so maybe less epochs will be better.

In addition to the works above, I also try to build a model with 3/4 of training data and 25 epochs(the work doesn't show in this notebook). But more data couldn't get higher score, too. So I think maybe the problem would be the number of features but not the number of epochs or training data.

#### For 500 features:
#### use 500 features and more training data

From the result above, it seems that using more features might be a good choice, too. Thus, I try to use 500 features with 500k data and 25 epochs to train the first 500 features model(the work doesn't show in this notebook). Surprisingly, the score doesn't improve but worse than 300 features with the same number of training data!

Since adjusting the number of features couldn't get higher score, I think more data might be another important fact, too. So, I use 3/4 of training data with 25 epochs to train second model(the work doesn't show in this notebook). This time I finally exceed the benchmark! This result shows that more data can help the model to find true attributes and get a link between these attributes and true labels.

The third try is training with more epochs in order to ensure that more epochs doesn't improve the score. I use another 25 epochs to train the model which has already trained by 3/4 data with 25 epochs(the work doesn't show in the no). Indeed, the result isn't higher than the first one.

The final try is training with less epochs. Below is the work for building the network and training it with only 9 epochs. It's almost the same as above work. The result is only higher a little bit than 25 epochs. Thus, it seems that the number of epochs doesn't affect a lot in this task.

In [91]:
import nltk
BOW_vectorizer = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)

In [92]:
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
num_of_train = int(len(train_df)*3/4)
BOW_vectorizer.fit(train_df['text'][:num_of_train])

# 2. Transform documents to document-term matrix.
X_train = BOW_vectorizer.transform(train_df['text'][:num_of_train])
y_train = train_df['emotion'][:num_of_train]
X_test = BOW_vectorizer.transform(train_df['text'][num_of_train:])
y_test = train_df['emotion'][num_of_train:]

In [93]:
import keras

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1091672, 500)
y_train.shape:  (1091672,)
X_test.shape:  (363891, 500)
y_test.shape:  (363891,)


In [95]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

In [96]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [97]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 9
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')
model.save('BOW500_1350k_e9.h5')

Train on 1091672 samples, validate on 363891 samples
Epoch 1/9
1091672/1091672 [==============================] - 105s 96us/step - loss: 1.4227 - acc: 0.4808 - val_loss: 1.3949 - val_acc: 0.4920
Epoch 2/9
1091672/1091672 [==============================] - 106s 97us/step - loss: 1.3791 - acc: 0.4959 - val_loss: 1.3849 - val_acc: 0.4968
Epoch 3/9
1091672/1091672 [==============================] - 107s 98us/step - loss: 1.3677 - acc: 0.5005 - val_loss: 1.3784 - val_acc: 0.4978
Epoch 4/9
1091672/1091672 [==============================] - 106s 97us/step - loss: 1.3611 - acc: 0.5026 - val_loss: 1.3765 - val_acc: 0.4982
Epoch 5/9
1091672/1091672 [==============================] - 107s 98us/step - loss: 1.3568 - acc: 0.5048 - val_loss: 1.3761 - val_acc: 0.4986
Epoch 6/9
1091672/1091672 [==============================] - 107s 98us/step - loss: 1.3535 - acc: 0.5058 - val_loss: 1.3745 - val_acc: 0.4996
Epoch 7/9
1091672/1091672 [==============================] - 106s 97us/step - loss: 1.3510 - ac

In [99]:
y_test_pred = model.predict(X_test)

In [100]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), label_decode(label_encoder,y_test_pred)), 2)))

c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


testing accuracy: 0.5


In [101]:
X_unknown = BOW_vectorizer.transform(test_df['text'])

In [102]:
y_unknown_pred = model.predict(X_unknown)

In [103]:
y_unknown_pred = label_decode(label_encoder, y_unknown_pred)

c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [104]:
predict_test = {
    "id":list(test_df['id']),
    "emotion":list(y_unknown_pred)
}

In [105]:
predict_test = pd.DataFrame(predict_test)
predict_test.to_csv('1350k_e9_submission.csv',index=False)

### 3. Conclusion
From all works above, we can get some conclusions. First, 500 features gets higher training accuracy at the first time than 300 features. That is, more features can include more useful informations. Second, more training data can really get higher score. Third, the number of epochs doesn't affect a lot in this competition.